In [9]:
import os
import sys
import json

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from data_module.data_module import FFTDataModule
from model.enc_classifier_model import AE1DClassifier

In [2]:
random_seed = 42
L.seed_everything(random_seed)

Seed set to 42


42

In [6]:
n_epochs = 20000
patience = n_epochs//100

optimizer_param_dict = {
    "Adam": (optim.Adam, {
        "lr": 0.001,
    }),
    "SGD": (optim.SGD, {
        "lr": 0.001,
        "momentum": 0.5,
    }),
}
batch_size = 512
optimizer, optimizer_param = optimizer_param_dict["Adam"]
dataset_path = "/nfs/ksdata/tran/HAR_AE/dataset/processed_concat_data"
AE1D_ckpt_path = "lightning_logs/14_AE_5classes/pattern_27/version_0/checkpoints/sample_epoch=3419-step=68400-val_loss=0.000000.ckpt"

log_save_dir = "lightning_logs/15_enc_classify_5classes_AEpattern27"

In [ ]:
print(" ----------------------start training---------------------------")
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger
from lightning.pytorch.utilities.model_summary import ModelSummary

def train_one_model(net, model_name):
    tensorboard_logger = TensorBoardLogger(save_dir=log_save_dir, name=model_name,)
    csv_logger = CSVLogger(save_dir=log_save_dir, name=model_name,)
    checkpoint_callback = ModelCheckpoint(
        dirpath=None,
        save_top_k=1,
        monitor="val_loss",
        mode="min",
        filename="sample_{epoch:02d}-{step:02d}-{val_loss:02f}"
    )
    
    trainer = L.Trainer(
        logger=[tensorboard_logger, csv_logger],
        callbacks=[EarlyStopping(monitor="val_loss", patience=patience), checkpoint_callback],
        max_epochs=n_epochs,
        check_val_every_n_epoch=10,
        accelerator="auto"
        )
    
    data_module = FFTDataModule(dataset_path=dataset_path, batch_size=batch_size)
    
    trainer.fit(model=net, datamodule=data_module)
    trainer_test_dict = trainer.logged_metrics
    
    trainer.test(model=net, datamodule=data_module)
    trainer_test_dict.update(trainer.logged_metrics)

    for key in trainer_test_dict.keys():
        trainer_test_dict[key] = trainer_test_dict[key].item()
    
    with open(os.path.join(trainer.logger.log_dir, "result.json"), "w") as f:
        json.dump(trainer_test_dict, f)

In [7]:
run_filter_list = [
    {"linear_channel_param": [128, 64], "is_dropout": False, },
    {"linear_channel_param": [64, 32], "is_dropout": False, },
    {"linear_channel_param": [128, 64], "is_dropout": True, },
    {"linear_channel_param": [64, 32], "is_dropout": True, },
]

In [10]:
import argparse

parser = argparse.ArgumentParser(description="", formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("--idx", type=int, default=0,
                    help=f"learning pattern min=0, max={len(run_filter_list) - 1}")

if os.path.basename(sys.argv[0]) == 'ipykernel_launcher.py':
    args = parser.parse_args(args=[])
else:
    args = parser.parse_args()

In [ ]:
cnn_filters, linear_filters = run_filter_list[args.idx]
net = AE1DClassifier(
    AE1D_ckpt_path = AE1D_ckpt_path,
    optimizer = optimizer,
    optimizer_param = optimizer_param, 
    out_features = 5,
    **run_filter_list[args.idx],
)
model_summary = ModelSummary(net, max_depth=6)
print(model_summary)

train_one_model(net, model_name=f"pattern_{args.idx}")